# Glicko-2 Reward Function for Nectar Dataset

This notebook implements a reward function based on Glicko-2 ratings for the Berkeley NEST Nectar dataset. The reward function is defined as:

```
Reward = Glicko-2 Rating - Rating Volatility
```

This rewards items with high ratings while penalizing those with high volatility (inconsistency).

In [1]:
# -*- coding: utf-8 -*-
import time
import math
from collections import defaultdict
from datetime import datetime
import pandas as pd
from datasets import load_dataset, IterableDataset # Make sure IterableDataset is imported if using streaming
from glicko2 import Player

In [2]:
dataset = load_dataset("stanfordnlp/shp", data_dir="askculinary", split='train')

In [3]:
start_time = time.time()

In [4]:
# --- 3. Initialize Player Storage ---
print("Initializing player storage...")
# Use the Player class defined above
players = {} # Dictionary to store author_id -> Player object instance
all_known_players = set() # Keep track of all players ever encountered

def get_player(author_id):
    """Gets or creates a Player object for a valid author ID."""
    # Check specifically against None and handle empty strings if necessary
    if author_id is not None and author_id.strip() != '' and author_id.lower() != '[deleted]':
        if author_id not in players:
            players[author_id] = Player() # Use the provided Player class
            # print(f"Created player: {author_id}") # Optional: for debugging
        all_known_players.add(author_id) # Track all valid players encountered
        return players.get(author_id)
    return None # Return None for invalid authors

Initializing player storage...


In [5]:
# --- 4. Process Matches and Group by Period ---
print("Processing matches and grouping by period...")
matches_by_period = defaultdict(lambda: defaultdict(lambda: {'ratings': [], 'rds': [], 'outcomes': []}))
processed_count = 0
# max_rows_to_process = 100000 # Optional: Limit rows for testing even without streaming
skipped_same_author = 0
skipped_deleted = 0
skipped_fetch_error = 0
max_rows_to_process = 5000
# Now iterate directly over the loaded dataset (which acts like a list/dict)
try:
    for i, row in enumerate(dataset):
        processed_count += 1
        # if processed_count > max_rows_to_process: # Apply optional limit if defined
        #      print(f"Reached processing limit of {max_rows_to_process} rows.")
        #      break

        # Extract data
        author_j = row.get('c_root_id_A')
        author_k = row.get('c_root_id_B')
        label = row.get('labels')
        timestamp_utc = row.get('created_at_utc_A') or row.get('created_at_utc_B')

        if author_j is None or author_k is None or label is None or timestamp_utc is None:
            skipped_fetch_error += 1
            continue
        if author_j == author_k:
            skipped_same_author += 1
            continue

        player_j = get_player(author_j)
        player_k = get_player(author_k)

        if player_j is None or player_k is None:
            skipped_deleted += 1
            continue

        try:
            period = datetime.utcfromtimestamp(int(timestamp_utc)).strftime('%Y-%m')
        except (ValueError, TypeError):
             period = "unknown-period"

        rating_j_current = player_j.rating
        rd_j_current = player_j.rd
        rating_k_current = player_k.rating
        rd_k_current = player_k.rd

        if label == 0: # Player J won
            outcome_for_j = 1.0
            outcome_for_k = 0.0
        elif label == 1: # Player K won
            outcome_for_j = 0.0
            outcome_for_k = 1.0
        else:
            outcome_for_j = 0.5 # Assume draw otherwise
            outcome_for_k = 0.5

        matches_by_period[period][author_j]['ratings'].append(rating_k_current)
        matches_by_period[period][author_j]['rds'].append(rd_k_current)
        matches_by_period[period][author_j]['outcomes'].append(outcome_for_j)

        matches_by_period[period][author_k]['ratings'].append(rating_j_current)
        matches_by_period[period][author_k]['rds'].append(rd_j_current)
        matches_by_period[period][author_k]['outcomes'].append(outcome_for_k)

        if processed_count % 20000 == 0: # Adjust print frequency as needed
            print(f"Gathered data for {processed_count}/{len(dataset)} rows...")

except Exception as e:
     print(f"An error occurred during data iteration at row {processed_count}: {e}")


print(f"\n--- Data Gathering Summary ---")
# ... [Summary print statements remain the same] ...
print(f"Total rows processed: {processed_count}")
print(f"Matches skipped (same author): {skipped_same_author}")
print(f"Matches skipped (deleted/invalid author): {skipped_deleted}")
print(f"Matches skipped (data fetch error): {skipped_fetch_error}")
print(f"Total unique valid authors encountered: {len(all_known_players)}")
print(f"Number of rating periods found: {len(matches_by_period)}")


Processing matches and grouping by period...
Gathered data for 20000/45710 rows...
Gathered data for 40000/45710 rows...

--- Data Gathering Summary ---
Total rows processed: 45710
Matches skipped (same author): 0
Matches skipped (deleted/invalid author): 0
Matches skipped (data fetch error): 0
Total unique valid authors encountered: 27733
Number of rating periods found: 132


In [6]:
# --- 5. Run Glicko-2 Updates Period by Period ---
print("\nUpdating ratings period by period...")
# ... [Rating update loop remains the same as previous response] ...
sorted_periods = sorted(matches_by_period.keys())

for period in sorted_periods:
    print(f"Processing period: {period}")
    players_in_period = set(matches_by_period[period].keys())

    # Identify players who *didn't* compete in this period but existed before
    inactive_players_in_period = all_known_players - players_in_period

    # Apply Step 6: Update RD for inactive players
    inactive_update_count = 0
    for author_id in inactive_players_in_period:
        player = players.get(author_id)
        if player: # Should always exist if in all_known_players
             try:
                 player.did_not_compete()
                 inactive_update_count += 1
             except Exception as e:
                 print(f"Error calling did_not_compete for {author_id} in period {period}: {e}")

    # Apply Steps 3-5: Update rating, RD, vol for active players
    updates_count = 0
    for author_id in players_in_period:
        player = players.get(author_id)
        if not player: continue # Should not happen if key exists, but safe check

        period_data = matches_by_period[period][author_id]
        opponent_ratings = period_data['ratings']
        opponent_rds = period_data['rds']
        outcomes = period_data['outcomes']

        # Ensure we have opponents before calling update
        if opponent_ratings:
            try:
                player.update_player(opponent_ratings, opponent_rds, outcomes)
                updates_count += 1
            except OverflowError:
                 print(f"OverflowError encountered updating player {author_id} in period {period}. Skipping update.")
                 # This can happen with extreme rating differences or RDs. Might indicate need for parameter tuning or data cleaning.
                 # Consider logging player state: player.rating, player.rd, player.vol and opponent data
            except FloatingPointError as fpe:
                print(f"FloatingPointError encountered updating player {author_id} in period {period}: {fpe}. Skipping update.")
            except Exception as e:
                 print(f"Error updating player {author_id} in period {period}: {e}")


    print(f"-> Completed updates for {updates_count} active players. Applied inactivity update for {inactive_update_count} players.")


print("Rating updates complete.")





Updating ratings period by period...
Processing period: 2012-02
-> Completed updates for 8 active players. Applied inactivity update for 27725 players.
Processing period: 2012-03
-> Completed updates for 22 active players. Applied inactivity update for 27711 players.
Processing period: 2012-04
-> Completed updates for 33 active players. Applied inactivity update for 27700 players.
Processing period: 2012-05
-> Completed updates for 54 active players. Applied inactivity update for 27679 players.
Processing period: 2012-06
-> Completed updates for 59 active players. Applied inactivity update for 27674 players.
Processing period: 2012-07
-> Completed updates for 39 active players. Applied inactivity update for 27694 players.
Processing period: 2012-08
-> Completed updates for 73 active players. Applied inactivity update for 27660 players.
Processing period: 2012-09
-> Completed updates for 100 active players. Applied inactivity update for 27633 players.
Processing period: 2012-10
-> Comp

In [7]:
# Setup players for test set
dataset_test = load_dataset("stanfordnlp/shp", data_dir="askbaking", split='train')

for i, row in enumerate(dataset_test):
        processed_count += 1
        # if processed_count > max_rows_to_process: # Apply optional limit if defined
        #      print(f"Reached processing limit of {max_rows_to_process} rows.")
        #      break

        # Extract data
        author_j = row.get('c_root_id_A')
        author_k = row.get('c_root_id_B')
        if author_j is None or author_k is None or label is None:
            skipped_fetch_error += 1
            continue
        if author_j == author_k:
            skipped_same_author += 1
            continue

        player_j = get_player(author_j)
        player_k = get_player(author_k)

In [8]:
# --- 6. Display Results ---
print("\n--- Final Glicko-2 Ratings for Authors ---")
# ... [Result display remains the same as previous response] ...
author_ratings = []
skipped_final_rating = 0
for author_id, player in players.items():
    try:
        # Add check for NaN/Inf before appending
        rating_val = player.rating
        rd_val = player.rd
        vol_val = player.vol
        if not (math.isfinite(rating_val) and math.isfinite(rd_val) and math.isfinite(vol_val)):
            print(f"Warning: Non-finite values for player {author_id}. Rating={rating_val}, RD={rd_val}, Vol={vol_val}. Skipping.")
            skipped_final_rating += 1
            continue

        author_ratings.append({
            "author": author_id,
            "rating": rating_val, # Uses the property getter
            "deviation (RD)": rd_val, # Uses the property getter
            "volatility": vol_val
        })
    except Exception as e:
        print(f"Error retrieving final rating for {author_id}: {e}")
        skipped_final_rating += 1


if skipped_final_rating > 0:
    print(f"\nNote: Skipped retrieving final ratings for {skipped_final_rating} authors due to errors or non-finite values.")

# Sort by rating descending for readability
results_df = pd.DataFrame(author_ratings)

if not results_df.empty:
    # Ensure columns are numeric before sorting
    results_df['rating'] = pd.to_numeric(results_df['rating'], errors='coerce')
    results_df['deviation (RD)'] = pd.to_numeric(results_df['deviation (RD)'], errors='coerce')
    results_df['volatility'] = pd.to_numeric(results_df['volatility'], errors='coerce')
    results_df.dropna(subset=['rating'], inplace=True) # Remove rows where rating became NaN

if not results_df.empty:
    results_df = results_df.sort_values(by="rating", ascending=False)

    # Display top N and bottom N for brevity, or the whole list
    pd.set_option('display.max_rows', 100) # Show more rows if needed
    print(f"Displaying top {min(20, len(results_df))} and bottom {min(10, len(results_df))} authors:")
    print(results_df.head(20))
    if len(results_df) > 30:
        print("...")
        print(results_df.tail(10))
    elif len(results_df) > 20:
         print("...") # Just indicate truncation if list is moderately long
else:
    print("No valid final ratings could be generated or displayed.")




--- Final Glicko-2 Ratings for Authors ---
Displaying top 20 and bottom 10 authors:
        author       rating  deviation (RD)  volatility
19637  g8prs9w  1992.480060      100.282789    0.062126
21477  fhxvbdb  1991.604800      105.601607    0.062003
21478  fhxvy7p  1990.857223      106.431689    0.061885
21479  fhxwecv  1990.068149      107.307655    0.061770
6271   egqq2c3  1988.954041      114.397174    0.061660
21480  fhxxmfo  1988.350874      109.209644    0.061555
19659  g8q20ur  1986.619006      107.192992    0.061355
6272   egqpx63  1985.046321      118.356740    0.061261
27525  edh401b  1985.019930      118.832233    0.061261
615    gb8rkg3  1983.268621      110.554002    0.061084
19715  g68rsv0  1982.569493      132.744482    0.060317
4458   g0h9e9v  1981.898526      113.495648    0.061001
4457   g0h6a07  1980.514749      115.017623    0.060922
4471   g0hb3ve  1979.024602      116.638843    0.060846
5921   ee2ojqo  1978.442948      125.358886    0.060846
18951  ec9lmlu  197

In [9]:
results_df.columns

Index(['author', 'rating', 'deviation (RD)', 'volatility'], dtype='object')

In [32]:
tr_auth = {}
for row in dataset:
    tr_auth[row['c_root_id_A']] = 1 + tr_auth.get(row['c_root_id_A'], 0)
    tr_auth[row['c_root_id_B']] = 1 + tr_auth.get(row['c_root_id_B'], 0)

In [28]:
ts_auth = {}
lim = 10
i = 0
for row in dataset_test:
    print(row['c_root_id_A'], row['c_root_id_A'] in tr_auth)
    ts_auth[row['c_root_id_A']] = 1 + ts_auth.get(row['c_root_id_A'], 0)
    ts_auth[row['c_root_id_B']] = 1 + ts_auth.get(row['c_root_id_B'], 0)
    if i > lim:
        break
    i += 1



ghbo9z3 False
ghbpodl False
ghbi7k3 False
ghbpodl False
ghbhsne False
ghbkei4 False
ghbpodl False
ghblc5e False
ghbkfv3 False
ghbnbik False
ghbolh4 False
ghbo9z3 False


In [33]:
cnt = 0 
for key in tr_auth:
    if key not in tr_auth:
        cnt += 1
print(cnt)

0


In [34]:
len(list(tr_auth.keys()))

27733

In [35]:
len(dataset)

45710

In [10]:
# Convert to DataFrame
results_df = pd.DataFrame(author_ratings)

# --- Normalization Step ---
if not results_df.empty:
    print(f"\nNormalizing {len(results_df)} valid results...")
    # Min-Max Normalization (scaling to [0, 1])

    # Rating
    r_min = results_df['rating'].min()
    r_max = results_df['rating'].max()
    if (r_max - r_min) != 0:
        results_df['rating_norm'] = (results_df['rating'] - r_min) / (r_max - r_min)
    else:
        results_df['rating_norm'] = 0.5 # Assign midpoint if all values are the same

    # Deviation (RD)
    rd_min = results_df['deviation (RD)'].min()
    rd_max = results_df['deviation (RD)'].max()
    if (rd_max - rd_min) != 0:
        results_df['rd_norm'] = (results_df['deviation (RD)'] - rd_min) / (rd_max - rd_min)
    else:
        results_df['rd_norm'] = 0.5

    # Volatility
    v_min = results_df['volatility'].min()
    v_max = results_df['volatility'].max()
    if (v_max - v_min) != 0:
        results_df['volatility_norm'] = (results_df['volatility'] - v_min) / (v_max - v_min)
    else:
        results_df['volatility_norm'] = 0.5

    # Sort by original rating
    results_df = results_df.sort_values(by="rating", ascending=False)

    # Display selected columns including normalized ones
    pd.set_option('display.max_rows', 100)
    pd.set_option('display.width', 120) # Adjust width for better display
    print(f"\nDisplaying top {min(20, len(results_df))} authors (incl. normalized scores):")
    # Select columns to display
    display_cols = ['author', 'rating', 'rating_norm', 'deviation (RD)', 'rd_norm', 'volatility', 'volatility_norm']
    print(results_df[display_cols].head(20).round(4)) # Round for display

    if len(results_df) > 20:
        print("...")
        print(f"\nDisplaying bottom {min(10, len(results_df))} authors (incl. normalized scores):")
        print(results_df[display_cols].tail(min(10, max(0, len(results_df)))).round(4))

else:
    print("No valid final ratings could be generated or displayed.")

# Optionally, save to CSV (including normalized columns)
try:
    if not results_df.empty:
        results_df.to_csv("askculinary_author_ratings_normalized.csv", index=False, float_format='%.6f')
        print("\nResults saved to askculinary_author_ratings_normalized.csv")
except Exception as e:
    print(f"\nError saving results to CSV: {e}")
end_time = time.time()
print(f"\nCurrent time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total execution time: {end_time - start_time:.2f} seconds")


Normalizing 41014 valid results...

Displaying top 20 authors (incl. normalized scores):
        author     rating  rating_norm  deviation (RD)  rd_norm  volatility  volatility_norm
19637  g8prs9w  1992.4801       1.0000        100.2828   0.0061      0.0621           1.0000
21477  fhxvbdb  1991.6048       0.9991        105.6016   0.0273      0.0620           0.9428
21478  fhxvy7p  1990.8572       0.9984        106.4317   0.0306      0.0619           0.8877
21479  fhxwecv  1990.0681       0.9977        107.3077   0.0340      0.0618           0.8346
6271   egqq2c3  1988.9540       0.9966        114.3972   0.0623      0.0617           0.7834
21480  fhxxmfo  1988.3509       0.9960        109.2096   0.0416      0.0616           0.7342
19659  g8q20ur  1986.6190       0.9943        107.1930   0.0336      0.0614           0.6413
6272   egqpx63  1985.0463       0.9928        118.3567   0.0780      0.0613           0.5976
27525  edh401b  1985.0199       0.9927        118.8322   0.0799      0.06

In [11]:
def create_reward_function(rating_coef=1.0, rd_coef=-0.5, volatility_coef=-0.2):
    """
    Creates a reward function that linearly combines normalized Glicko-2 metrics.
    
    Args:
        rating_coef (float): Coefficient for normalized rating (default: 1.0)
                             Higher values prioritize higher skill ratings
        rd_coef (float): Coefficient for normalized rating deviation (default: -0.5)
                         Negative value penalizes uncertainty in rating
        volatility_coef (float): Coefficient for normalized volatility (default: -0.2)
                                Negative value penalizes inconsistent performance
    
    Returns:
        function: A reward function that takes a row from results_df and returns a scalar reward
    """
    def reward_function(row):
        """
        Calculates reward for a given author based on their Glicko-2 metrics.
        
        Args:
            row: A row from results_df containing normalized Glicko-2 metrics
            
        Returns:
            float: Calculated reward score
        """
        # Linear combination of normalized metrics
        reward = (
            rating_coef * row['rating_norm'] +
            rd_coef * row['rd_norm'] +
            volatility_coef * row['volatility_norm']
        )
        
        # Ensure reward is non-negative (optional, depending on your requirements)
        # reward = max(0.0, reward)
        
        return reward
    
    return reward_function


# Example usage in a GRPOTrainer context:
def setup_grpo_trainer(results_df, rating_coef=1.0, rd_coef=-0.5, volatility_coef=-0.2):
    """
    Sets up a GRPOTrainer with a reward function based on Glicko-2 metrics.
    
    Args:
        results_df (pd.DataFrame): DataFrame containing normalized Glicko-2 metrics
        rating_coef, rd_coef, volatility_coef: Hyperparameters for the reward function
        
    Returns:
        The configured GRPOTrainer
    """
    # Create the reward function with specified coefficients
    reward_func = create_reward_function(
        rating_coef=rating_coef,
        rd_coef=rd_coef,
        volatility_coef=volatility_coef
    )
    
    # Apply the reward function to get reward scores
    results_df['reward_score'] = results_df.apply(reward_func, axis=1)
    
    # Here you would initialize and configure your GRPOTrainer
    # using the reward scores in results_df
    # This is just a placeholder as the actual GRPO implementation would depend on your framework
    # grpo_trainer = GRPOTrainer(reward_function=reward_func, ...)
    
    return results_df  # Return the updated DataFrame with reward scores
    # In a real implementation, you would return the trainer instead

In [12]:



# --- 5. Run Glicko-2 Updates Period by Period ---
print("\nUpdating ratings period by period...")
# ... [Rating update loop remains the same as previous response] ...
sorted_periods = sorted(matches_by_period.keys())

for period in sorted_periods:
    print(f"Processing period: {period}")
    players_in_period = set(matches_by_period[period].keys())

    # Identify players who *didn't* compete in this period but existed before
    inactive_players_in_period = all_known_players - players_in_period

    # Apply Step 6: Update RD for inactive players
    inactive_update_count = 0
    for author_id in inactive_players_in_period:
        player = players.get(author_id)
        if player: # Should always exist if in all_known_players
             try:
                 player.did_not_compete()
                 inactive_update_count += 1
             except Exception as e:
                 print(f"Error calling did_not_compete for {author_id} in period {period}: {e}")

    # Apply Steps 3-5: Update rating, RD, vol for active players
    updates_count = 0
    for author_id in players_in_period:
        player = players.get(author_id)
        if not player: continue # Should not happen if key exists, but safe check

        period_data = matches_by_period[period][author_id]
        opponent_ratings = period_data['ratings']
        opponent_rds = period_data['rds']
        outcomes = period_data['outcomes']

        # Ensure we have opponents before calling update
        if opponent_ratings:
            try:
                player.update_player(opponent_ratings, opponent_rds, outcomes)
                updates_count += 1
            except OverflowError:
                 print(f"OverflowError encountered updating player {author_id} in period {period}. Skipping update.")
                 # This can happen with extreme rating differences or RDs. Might indicate need for parameter tuning or data cleaning.
                 # Consider logging player state: player.rating, player.rd, player.vol and opponent data
            except FloatingPointError as fpe:
                print(f"FloatingPointError encountered updating player {author_id} in period {period}: {fpe}. Skipping update.")
            except Exception as e:
                 print(f"Error updating player {author_id} in period {period}: {e}")


    print(f"-> Completed updates for {updates_count} active players. Applied inactivity update for {inactive_update_count} players.")


print("Rating updates complete.")


# --- 6. Display Results ---
print("\n--- Final Glicko-2 Ratings for Authors ---")
# ... [Result display remains the same as previous response] ...
author_ratings = []
skipped_final_rating = 0
for author_id, player in players.items():
    try:
        # Add check for NaN/Inf before appending
        rating_val = player.rating
        rd_val = player.rd
        vol_val = player.vol
        if not (math.isfinite(rating_val) and math.isfinite(rd_val) and math.isfinite(vol_val)):
            print(f"Warning: Non-finite values for player {author_id}. Rating={rating_val}, RD={rd_val}, Vol={vol_val}. Skipping.")
            skipped_final_rating += 1
            continue

        author_ratings.append({
            "author": author_id,
            "rating": rating_val, # Uses the property getter
            "deviation (RD)": rd_val, # Uses the property getter
            "volatility": vol_val
        })
    except Exception as e:
        print(f"Error retrieving final rating for {author_id}: {e}")
        skipped_final_rating += 1


if skipped_final_rating > 0:
    print(f"\nNote: Skipped retrieving final ratings for {skipped_final_rating} authors due to errors or non-finite values.")

# Sort by rating descending for readability
results_df = pd.DataFrame(author_ratings)

if not results_df.empty:
    # Ensure columns are numeric before sorting
    results_df['rating'] = pd.to_numeric(results_df['rating'], errors='coerce')
    results_df['deviation (RD)'] = pd.to_numeric(results_df['deviation (RD)'], errors='coerce')
    results_df['volatility'] = pd.to_numeric(results_df['volatility'], errors='coerce')
    results_df.dropna(subset=['rating'], inplace=True) # Remove rows where rating became NaN

if not results_df.empty:
    results_df = results_df.sort_values(by="rating", ascending=False)

    # Display top N and bottom N for brevity, or the whole list
    pd.set_option('display.max_rows', 100) # Show more rows if needed
    print(f"Displaying top {min(20, len(results_df))} and bottom {min(10, len(results_df))} authors:")
    print(results_df.head(20))
    if len(results_df) > 30:
        print("...")
        print(results_df.tail(10))
    elif len(results_df) > 20:
         print("...") # Just indicate truncation if list is moderately long
else:
    print("No valid final ratings could be generated or displayed.")


# Optionally, save to CSV
# try:
#     if not results_df.empty:
#         results_df.to_csv("askculinary_author_ratings_custom_glicko_full.csv", index=False)
#         print("\nResults saved to askculinary_author_ratings_custom_glicko_full.csv")
# except Exception as e:
#     print(f"\nError saving results to CSV: {e}")


end_time = time.time()
print(f"\nCurrent time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Total execution time: {end_time - start_time:.2f} seconds")


Updating ratings period by period...
Processing period: 2012-02
-> Completed updates for 8 active players. Applied inactivity update for 41006 players.
Processing period: 2012-03
-> Completed updates for 22 active players. Applied inactivity update for 40992 players.
Processing period: 2012-04
-> Completed updates for 33 active players. Applied inactivity update for 40981 players.
Processing period: 2012-05
-> Completed updates for 54 active players. Applied inactivity update for 40960 players.
Processing period: 2012-06
-> Completed updates for 59 active players. Applied inactivity update for 40955 players.
Processing period: 2012-07
-> Completed updates for 39 active players. Applied inactivity update for 40975 players.
Processing period: 2012-08
-> Completed updates for 73 active players. Applied inactivity update for 40941 players.
Processing period: 2012-09
-> Completed updates for 100 active players. Applied inactivity update for 40914 players.
Processing period: 2012-10
-> Comp

In [13]:
dataset_test = load_dataset("stanfordnlp/shp", data_dir="askbaking")

In [14]:
# --- 1. Provided Glicko-2 Player Class ---
class Player:
    # Class attribute
    # The system constant, which constrains
    # the change in volatility over time.
    _tau = 0.5

    def getRating(self):
        return (self.__rating * 173.7178) + 1500

    def setRating(self, rating):
        self.__rating = (rating - 1500) / 173.7178

    rating = property(getRating, setRating)

    def getRd(self):
        return self.__rd * 173.7178

    def setRd(self, rd):
        self.__rd = rd / 173.7178

    rd = property(getRd, setRd)

    def __init__(self, rating = 1500, rd = 350, vol = 0.06):
        # For testing purposes, preload the values
        # assigned to an unrated player.
        self.setRating(rating)
        self.setRd(rd)
        self.vol = vol

    def _preRatingRD(self):
        """ Calculates and updates the player's rating deviation for the
        beginning of a rating period.

        preRatingRD() -> None

        """
        self.__rd = math.sqrt(math.pow(self.__rd, 2) + math.pow(self.vol, 2))

    def update_player(self, rating_list, RD_list, outcome_list):
        """ Calculates the new rating and rating deviation of the player.

        update_player(list[int], list[int], list[bool]) -> None
        Docstring notes list[bool] for outcome, but expects numeric 0, 0.5, 1
        """
        # Convert the opponent rating and rating deviation values for internal use.
        rating_list = [(x - 1500) / 173.7178 for x in rating_list]
        RD_list = [x / 173.7178 for x in RD_list]

        v = self._v(rating_list, RD_list)
        self.vol = self._newVol(rating_list, RD_list, outcome_list, v)
        self._preRatingRD() # Applies Step 2 (modified RD) internally *after* vol update

        self.__rd = 1 / math.sqrt((1 / math.pow(self.__rd, 2)) + (1 / v))

        tempSum = 0
        for i in range(len(rating_list)):
            tempSum += self._g(RD_list[i]) * \
                       (outcome_list[i] - self._E(rating_list[i], RD_list[i]))
        self.__rating += math.pow(self.__rd, 2) * tempSum


    def _newVol(self, rating_list, RD_list, outcome_list, v):
        """ Calculating the new volatility as per the Glicko2 system.

        _newVol(list, list, list) -> float

        """
         # Convergence tolerance for the iteration
        CONVERGENCE_TOLERANCE = 0.000001
        i = 0
        delta = self._delta(rating_list, RD_list, outcome_list, v)
        a = math.log(math.pow(self.vol, 2))
        tau = self._tau
        # Use rating deviation (phi) from the start of the period for calculation
        pre_rating_rd_sq = math.pow(self.__rd, 2) + math.pow(self.vol, 2) # RD^2 + vol^2 before update

        # Simplified conditions for the Illinois algorithm based on Glickman's paper
        A = a
        if (delta**2 > pre_rating_rd_sq + v):
             B = math.log(delta**2 - pre_rating_rd_sq - v)
        else:
             k = 1
             while self._f(a - k * tau, delta, v, a, tau) < 0:
                  k = k + 1
             B = a - k * tau

        fA = self._f(A, delta, v, a, tau)
        fB = self._f(B, delta, v, a, tau)

        while math.fabs(B - A) > CONVERGENCE_TOLERANCE:
            C = A + (A - B) * fA / (fB - fA)
            fC = self._f(C, delta, v, a, tau)
            if fC * fB < 0:
                 A = B
                 fA = fB
            else:
                 fA = fA / 2.0
            B = C
            fB = fC

        return math.exp(A / 2.0)

    # Helper function for the volatility calculation's iterative method
    def _f(self, x, delta, v, a, tau):
        ex = math.exp(x)
        # Use rating deviation (phi) from the start of the period for calculation
        pre_rating_rd_sq = math.pow(self.__rd, 2) + math.pow(self.vol, 2) # RD^2 + vol^2 before update
        d_sq = pre_rating_rd_sq + v + ex # Denominator squared term estimate

        # Handle potential division by zero or log of non-positive if d_sq is problematic
        if d_sq <= 0: return -1 # Or some other indicator of error/boundary condition

        term1 = ex * (delta**2 - pre_rating_rd_sq - v - ex) / (2 * d_sq**2) if d_sq else 0
        term2 = (x - a) / tau**2
        return term1 - term2


    def _delta(self, rating_list, RD_list, outcome_list, v):
        """ The delta function of the Glicko2 system.

        _delta(list, list, list) -> float

        """
        tempSum = 0
        for i in range(len(rating_list)):
            tempSum += self._g(RD_list[i]) * (outcome_list[i] - self._E(rating_list[i], RD_list[i]))
        return v * tempSum

    def _v(self, rating_list, RD_list):
        """ The v function of the Glicko2 system.

        _v(list[int], list[int]) -> float

        """
        tempSum = 0
        for i in range(len(rating_list)):
            tempE = self._E(rating_list[i], RD_list[i])
            tempSum += math.pow(self._g(RD_list[i]), 2) * tempE * (1 - tempE)

        # Avoid division by zero if tempSum is zero (e.g., no opponents)
        if tempSum == 0:
            # Handle this case: perhaps return a very large number or raise error
            # Glickman's paper suggests V -> infinity, so 1/V -> 0.
            # Returning a very small inverse V, but update_player uses 1/V. Let's return a large V.
             return float('inf') # Or a very large number
        return 1 / tempSum

    def _E(self, p2rating, p2RD):
        """ The Glicko E function.

        _E(int) -> float

        """
        return 1 / (1 + math.exp(-1 * self._g(p2RD) * \
                                   (self.__rating - p2rating)))

    def _g(self, RD):
        """ The Glicko2 g(RD) function.

        _g() -> float

        """
        return 1 / math.sqrt(1 + 3 * math.pow(RD, 2) / math.pow(math.pi, 2))

    def did_not_compete(self):
        """ Applies Step 6 of the algorithm. Use this for
        players who did not compete in the rating period.

        did_not_compete() -> None

        """
        self._preRatingRD()